# LaMassa S82 (x-ray) catalog *versus* UKIDSS

In this notebook the cross-matching procedure applied by [LaMassa et al. 2016] (LM hereafter) to multi-wavelength catalogs will  be reproduced. Particularly, we will cross-match the x-ray data from XMM-Newton AO 13 cycle to UKIDSS near-infrared. The goal here is to verify whether we can recover the very same results therein, showing us our algorithm for Maximum Likelihood Estimator to be correct.

The UKIDSS catalog here in use is the same used in LM: UKIDSS-LAS Data Release 8, *primary* objects, cleaned from spurious/noisy detections; "?*apermag3*" magnitude measurements and accordingly errors were retrieved. The table file `ukidss.fits` here used can be taken from [my github repository][github_ukidss].

The LaMassa catalog --from where we'll take the x-ray sources-- was downloaded from [CDS].

[LaMassa et al. 2016]: http://arxiv.org/abs/1510.00852
[github_ukidss]: https://github.com/chbrandt/uks82
[CDS]: http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=J%2FApJ%2F817%2F172

# Maximum Likelihood Estimator (MLE)

MLE is applied by LM to find the correct --or most possible-- counterpart to their x-ray sources. MLE was first proposed by [Sutherland & Saunders in 1992] and is being adopted as a better alternative to the simplistic *nearest-neighbour* algorithm.

What MLE does is to estimate how probable a given counterpart candidate is to be real counterpart from a source in its vicinity. The method was developed having in mind that multiple candidates can be nearby in the (RA,Dec) sky-projected plan. Accordingly, the method includes the ancillary magnitudes as a third component to help differentiating background objects from candidate(s).

Consider the situation where there is a source "S" (which was observed by instrument "A") and in the vicinities, within a distance "da", of S there are $N$ objects ("N1", "N2", ..., "NN") that were observed by a different instrument ("B"). Also observed using "B", but distant a bit further from "S". there are $M$ objects ("M1", "M2", ..., "MM") that can not be related to "S", but will be of our help further on. The "M" objects lie beyond the distance "da" and before distance "db", and $db > da$. *We want now to answer the following question: which of the objects observed by "B" is in fact "S" (observed by a different instrument)?* Before coming with any answer, we are taught that instruments "A" and "B" suffer from different physical effects that lead to uncorrelated errors and different image resolutions when registering the pictures; which means that "S" and its (true) counterpart may *not* be one over the other, but shifted by some amount.

The distance "da" from "S" is considered to be "vicinity", and objects inside this distance are considered, *a priori*, candidates to the (true) counterpart. Such objects will be called *ancillary*  objects. The objects from sample `M` will be called *background* objects, they compose the sample of objects observed by "B" definitely *not* candidate to be "S" counterpart.

The MLE method will eventually give a score called *Reliability* ($R$) to each of the candidates. Such score --reliability-- is the probability of being the true counterpart, and is given by:
$$
R_j = \frac{LR_j}{\sum_j{LR_j}+(1-Q)}
$$

The central figure in MLE is the likelihood ratio, $LR$:
$$
LR_j = \frac{q(m) f(r)}{n(m)}
$$
. 

$f(r)$ is the prior regarding the position of the candidate object relative to the source. Typically, $f(r)$ is modelled as a bidimensional Gaussian with $\sigma$ being the quadrature sum of source's positional error and objects' average positional error:
$$
f(r) = \frac{1}{2 \pi \sigma} exp^{-r^2/2\sigma^2} ; 
$$
$$
\sigma = \frac{1}{2}\Big[\sqrt{\sigma^2_{\alpha_S} + \sigma^2_{\delta_S}} + \sqrt{\sigma^2_{\alpha_O} + \sigma^2_{\delta_O}}\Big]
$$

$q(m)$ is the likelihood of the object being a (good) candidate given its magnitude. It is computed by drawing the ancillary objects normalized magnitude distribution and subtracting from it the normalized background magnitude distribution.

Finally, $n(m)$ is the surface density of background objects with magnitude $m$.It is computed by counting the number of background objects per magnitude bin per square-degree; normalized by the number of objects.


[Sutherland & Saunders in 1992]: http://mnras.oxfordjournals.org/cgi/doi/10.1093/mnras/259.3.413


## The algorithm

Let's put it all together to build an algorithm.

To compute MLE quantities we need to define the background and ancillary samples. 
To do that we have to define the *search radius* ($r_s$) --from where the *ancillary* sample will come out-- and the *inner & outer radii* ($r_i$, $r_o$) for the background sample.

### Search radii

There are different ways to estimate the (best) *search radius*. 
Typically, the instrument's (nominal) error radius, systematic plus statistical, is used, as [LaMassa et al. 2016]. 
[Timlin et al. 2016] have used the Rayleigh Criterion to estimate such radius, considering then a physical limitation on resolving close by objects; similarly, the overall PSF (FWHM) is a valid estimator. 
Another way of estimating $r_s$, data driven, is by directly estimating the typical distance between the objects in each catalog.

Analogously, we have to define the *inner and outer radii*, from the primary source, of the annulus defining the background region. 
Trully speaking, the background region does *not* need to be drawn as an annulus centered centered in the source, but that is a straightforward, generic choice for sampling background sources.
It is important to notice that the background region should avoid other sources' ancillary sample, which is to say that the (annulus) region should not intersect with another source's search area.

* Estimate samples radii
  * (ancillary) search radius: $r_{s}$
  * background (annulus) radii: $r_{i} \lt r_{o}$


### Samples definition

Once we have the radii defined we cross-match the catalogs to define the *ancillary* and *background* samples; At this point, each source has two lists of objects related to it:

* Source
  * ancillary sample (within Rs)
  * background sample (between Ri and Ro)
  
But before looping through each primary source, we may define $q(m)$ and $n(m)$ as they are globally defined functions. And after we have $q(m)$ we may estimate $Q$.

* Estimate magnitude distributions
  * $n(m)$: background surface brightness distribution
  * $q(m)$: ancillary brightness distribution
  * $Q$: expected counterpart recover rate
  
  
#### Radial prior

The radial profile $f(r) \propto \sigma^{-1} \exp^{-r^2/\sigma^2}$ is ideally defined for each source, for $\sigma$ is a function of the source' and ancillary objects' positional errors, $\sigma_s$ and $\sigma_o$, resp.:
$$
\sigma = \sqrt{\frac{\sigma_s^2 + \sigma_o^2}{2}}
$$

If the positional errors are well behaved --i.e, their dispersion is small--, we may approximate $f(r)$ as a global function.
We may consider $\sigma_s$ and $\sigma_o$ as the mean of the respective positional errors.

* Compute mean positional errors
  * primary sources catalog
  * ancillary objects
* define $f(r)$


#### Likelood Ratio threshold

The LR-threshold, $LR_{th}$, is the minimum value an ancillary object may score to be considered a counterpart candidate.
There are different ways to compute $LR_{th}$, the simplest one is based on the reliability parameter in a assintotic case: consider there is only *one* ancillary object within the *search radius* around a source; in this case we would expect such object to be the true source' counterpart.
Considering the Reliability parameter, $R$ a probability score, $R_j=0.5$ is the minimal (reasonable) value for such parameter so that the object can be considered a candidate.
Using the definition of $R$ above we should have:
$$
0.5 = \frac{LR_{th}}{LR_{th} + (1-Q)}
$$

$$
LR_{th} = \frac{0.5(Q-1)}{-0.5}
$$

$$
LR_{th} = 1-Q
$$


### Counterpart evaluation

Now that we have all the ingredients in place we may visit each primary source' neighbourhood and evaluate each ancillary object.

For each source,
* Loop over the respective ancillary sample:
  * evaluate each object's $LR$
  * remove objects with $LR_j < LR_{th}$
* Sum all ancillaries' $LR_j$
* Loop over all candidates:
  * compute $R_j$
  
The highest $R_j$ is said to be the true counterpart.





[Timlin et al. 2016]: http://arxiv.org/abs/1603.08488

# Coding

The workflow has being worked out in `mle_implementation`. Looks like we have a close enough to LaMassa 2016+ implementation: approximately 5% are mismatched (see `mle_implementation_verification`).

We now go ahead to design code block, a module, for general use of this algorithm.
Eventually, we will merge this method to `booq/xmatch`.

### X-Ray (Lamassa-XMM-AO13)

In [1]:
import booq
from booq import table

tab_lm = table.ATable.read('lamassa/cds/xmmao13.dat',readme='lamassa/cds/ReadMe',format='cds')

In [2]:
df_lm = tab_lm.to_dataframe()
del tab_lm
df_lm.head()

,Seq,ObsID,RAdeg,DEdeg,e_Pos,DistNN,ExtFlag,InXMM,InChandra,FSoft,...,rH,F250,e_F250,F350,e_F350,F500,e_F500,XMMAO10CP,ChCP,CPCoord
0,2359.0,742830101.0,14.097,0.166,4.7,515.8,0.0,no,no,1.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2360.0,742830101.0,14.115,-0.353,3.5,171.7,0.0,no,no,1.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2361.0,742830101.0,14.115,-0.160,5.8,291.7,0.0,no,no,2.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2362.0,742830101.0,14.142,-0.442,3.4,49.1,0.0,no,no,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,2363.0,742830101.0,14.154,-0.448,2.1,49.1,0.0,no,no,7.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [3]:
ID_column_lm = 'Seq'
df_lm[ID_column_lm] = df_lm[ID_column_lm].astype(int).astype(str)

df_lm.head()

,Seq,ObsID,RAdeg,DEdeg,e_Pos,DistNN,ExtFlag,InXMM,InChandra,FSoft,...,rH,F250,e_F250,F350,e_F350,F500,e_F500,XMMAO10CP,ChCP,CPCoord
0,2359,742830101.0,14.097,0.166,4.7,515.8,0.0,no,no,1.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2360,742830101.0,14.115,-0.353,3.5,171.7,0.0,no,no,1.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2361,742830101.0,14.115,-0.160,5.8,291.7,0.0,no,no,2.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2362,742830101.0,14.142,-0.442,3.4,49.1,0.0,no,no,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,2363,742830101.0,14.154,-0.448,2.1,49.1,0.0,no,no,7.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


### IR (UKIDSS-DR8)

In [4]:
from booq.io import fits
uk = fits.open('ukidss/ukidss.fits')
uk.info


  file: ukidss/ukidss.fits
  extension: 1
  type: BINARY_TBL
  rows: 3501552
  column info:
    SOURCEID            i8  
    RA                  f8  
    DEC                 f8  
    SIGRA               f4  
    SIGDEC              f4  
    EPOCH               f8  
    EBV                 f4  
    AY                  f4  
    AJ                  f4  
    AH                  f4  
    AK                  f4  
    MERGEDCLASS         i2  
    MERGEDCLASSSTAT     f4  
    YHALLMAG            f4  
    YHALLMAGERR         f4  
    YPETROMAG           f4  
    YPETROMAGERR        f4  
    YPSFMAG             f4  
    YPSFMAGERR          f4  
    YAPERMAG3           f4  
    YAPERMAG3ERR        f4  
    J_1HALLMAG          f4  
    J_1HALLMAGERR       f4  
    J_1PETROMAG         f4  
    J_1PETROMAGERR      f4  
    J_1PSFMAG           f4  
    J_1PSFMAGERR        f4  
    J_1APERMAG3         f4  
    J_1APERMAG3ERR      f4  
    HHALLMAG            f4  
    HHALLMAGERR         f4  
    HPET

In [5]:
# cat_uk = table.read('ukidss/ukidss.fits',
#                     columns=['SOURCEID','RA','DEC','SIGRA','SIGDEC',
#                              'YAPERMAG3','J_1APERMAG3','HAPERMAG3','KAPERMAG3']
#                    )
# cat_uk['SOURCEID'] = cat_uk['SOURCEID'].astype(int).astype(str)

In [6]:
from booq import table
tab_uk_pos = table.ATable.read('ukidss/ukidss.fits',columns=['RA','DEC'])
#TODO: at init/reading be able to specific specific region(s) to load
#cat_uk = table.ATable.read('ukidss/ukidss.fits',conesearch=(ra,dec,radius),
#                          columns={'ra':'RA2000','dec':'DE2000'})
#cat_uk = table.ATable.read('ukidss/ukidss.fits',around=(cat_lm,radius),
#                          columns={'ra':'RA2000','dec':'DE2000'})
#cat_uk = table.ATable.read('ukidss/ukidss.fits',region=booq.region.Region,
#                          columns={'ra':'RA2000','dec':'DE2000'})

In [7]:
df_uk_pos = tab_uk_pos.to_dataframe()
del tab_uk_pos
df_uk_pos.head()

,RA,DEC
0,6.021207,0.021641
1,6.022647,0.021643
2,6.021880,0.021649
3,6.021893,0.021659
4,6.022186,0.021660


**For some reason, UKIDSS positional columns (RA,Dec) are in radians; transform to degrees**

In [8]:
from astropy.coordinates import Angle
ra = Angle(df_uk_pos['RA'].values,'rad').to('deg')
dec= Angle(df_uk_pos['DEC'].values,'rad').to('deg')
df_uk_pos['RA'] = ra
df_uk_pos['DEC'] = dec

df_uk_pos.describe()

                 RA           DEC
count  3.501552e+06  3.501552e+06
mean   1.841558e+02 -3.185555e-02
std    1.530424e+02  7.262582e-01
min    1.004211e-05 -1.249998e+00
25%    3.243559e+01 -6.691321e-01
50%    3.134817e+02 -5.326323e-02
75%    3.349581e+02  6.023953e-01
max    3.600000e+02  1.250000e+00

Total number of rows: 3501552 (3.5e+06)

-> Has Nil? (How many?)
RA     0
DEC    0
dtype: int64


#### Define a sub sample

Because the UKIDSS catalog is quite big, we should define a sub-sample to better explorer the data.

In [9]:
ra_min = df_lm.RAdeg.min() - 0.5
ra_max = df_lm.RAdeg.max() + 0.5
dec_min = df_lm.DEdeg.min() - 0.1
dec_max = df_lm.DEdeg.max() + 0.1
df_uk_sample = df_uk_pos.crop(RA=(ra_min,ra_max),
                           DEC=(dec_min,dec_max))
del df_uk_pos

df_uk_sample.describe()

                  RA            DEC
count  231338.000000  231338.000000
mean       21.215457       0.013751
std         4.365869       0.389857
min        13.597006      -0.721999
25%        17.474631      -0.301291
50%        21.748381       0.031961
75%        24.765722       0.342271
max        28.543990       0.703997

Total number of rows: 231338 (2.3e+05)

-> Has Nil? (How many?)
RA     0
DEC    0
dtype: int64


In [10]:
df_uk_sample.head()

,RA,DEC
601783,23.367877,0.687888
601784,23.460531,0.688232
601785,23.414002,0.687667
601786,23.280570,0.687302
601787,23.474184,0.687547


Now we get the positional-index of those selected lines to load the whole UKIDSS catalog.

In [11]:
# 'rows' contains the positions of our interest (the ones in df_uk_sample aget df_uk)
rows = df_uk_sample.index.values
del df_uk_sample

tab_uk_sample = table.ATable.read('ukidss/ukidss.fits',rows=rows)
tab_uk_sample

SOURCEID,RA,DEC,SIGRA,SIGDEC,EPOCH,EBV,AY,AJ,AH,AK,MERGEDCLASS,MERGEDCLASSSTAT,YHALLMAG,YHALLMAGERR,YPETROMAG,YPETROMAGERR,YPSFMAG,YPSFMAGERR,YAPERMAG3,YAPERMAG3ERR,J_1HALLMAG,J_1HALLMAGERR,J_1PETROMAG,J_1PETROMAGERR,J_1PSFMAG,J_1PSFMAGERR,J_1APERMAG3,J_1APERMAG3ERR,HHALLMAG,HHALLMAGERR,HPETROMAG,HPETROMAGERR,HPSFMAG,HPSFMAGERR,HAPERMAG3,HAPERMAG3ERR,KHALLMAG,KHALLMAGERR,KPETROMAG,KPETROMAGERR,KPSFMAG,KPSFMAGERR,KAPERMAG3,KAPERMAG3ERR
int64,float64,float64,float32,float32,float64,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
433827929919,0.407846390194,0.0120059039264,-9.99999e+08,-9.99999e+08,2005.90215116,0.0220508,0.0267036,0.0196032,0.0127454,0.0079383,1,4.93475,19.8162,0.339393,19.7927,0.352423,-9.99999e+08,-9.99999e+08,19.6622,0.129669,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,18.5194,0.341595,18.6227,0.444583,-9.99999e+08,-9.99999e+08,18.5062,0.151259,16.8393,0.166297,17.0132,0.244477,-9.99999e+08,-9.99999e+08,17.5441,0.124013
433827929920,0.409463516869,0.012011917063,-9.99999e+08,-9.99999e+08,2005.90215116,0.0236491,0.028639,0.021024,0.0136692,0.00851367,-1,0.421079,17.9835,0.0401669,17.9463,0.0584219,-9.99999e+08,-9.99999e+08,17.8783,0.0279219,17.4947,0.0500438,17.5421,0.0748066,-9.99999e+08,-9.99999e+08,17.3798,0.0304339,16.9268,0.0543857,16.9449,0.0858596,-9.99999e+08,-9.99999e+08,16.8934,0.035704,16.618,0.0808161,16.6613,0.118318,-9.99999e+08,-9.99999e+08,16.5653,0.0519861
433827929921,0.408651420325,0.0120020568056,-9.99999e+08,-9.99999e+08,2005.90215116,0.0226826,0.0274686,0.0201648,0.0131105,0.00816573,1,2.94201,19.834,0.366881,19.834,0.366881,-9.99999e+08,-9.99999e+08,20.2877,0.228542,19.2812,0.234339,19.2812,0.234339,-9.99999e+08,-9.99999e+08,19.6216,0.228677,18.7383,0.383572,18.7283,0.389533,-9.99999e+08,-9.99999e+08,18.6331,0.170696,17.6925,0.398679,17.6925,0.398679,-9.99999e+08,-9.99999e+08,17.7641,0.152736
433827929922,0.406322596785,0.0119956787088,-9.99999e+08,-9.99999e+08,2005.90215116,0.0225056,0.0272542,0.0200075,0.0130082,0.00810201,1,16.0753,17.4584,0.0395172,17.2767,0.0590367,-9.99999e+08,-9.99999e+08,17.841,0.0268707,16.9499,0.0415769,16.7474,0.0635827,-9.99999e+08,-9.99999e+08,17.2757,0.0275967,16.1115,0.0399464,16.0146,0.0600487,-9.99999e+08,-9.99999e+08,16.5572,0.0260672,15.3375,0.0384352,15.0854,0.0602332,-9.99999e+08,-9.99999e+08,15.8079,0.0255751
433827929923,0.409701804291,0.0119999505893,-9.99999e+08,-9.99999e+08,2005.90215116,0.0239274,0.0289761,0.0212715,0.0138301,0.00861388,1,0.782843,20.1834,0.387817,20.1834,0.387817,-9.99999e+08,-9.99999e+08,20.3265,0.239728,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08,-9.99999e+08
433827929924,0.409192913378,0.0120002894351,-9.99999e+08,-9.99999e+08,2005.90215116,0.0233148,0.0282342,0.0207268,0.0134759,0.00839331,1,15.8572,18.3361,0.11912,18.4298,0.200747,-9.99999e+08,-9.99999e+08,18.941,0.0686888,17.7032,0.105149,17.7141,0.127878,-9.99999e+08,-9.99999e+08,18.4453,0.0785382,17.0186,0.134257,17.1364,0.218883,-9.99999e+08,-9.99999e+08,17.5933,0.0666682,16.3074,0.130962,16.4882,0.237619,-9.99999e+08,-9.99999e+08,16.8477,0.0668407
433827929925,0.407144874736,0.0119879010813,-9.99999e+08,-9.99999e+08,2005.90215116,0.0219588,0.0265921,0.0195214,0.0126922,0.00790517,1,10.203,18.5988,0.166848,18.6272,0.239986,-9.99999e+08,-9.99999e+08,19.2887,0.0926362,18.2093,0.111538,18.1836,0.116781,-9.99999e+08,-9.99999e+08,19.0402,0.133803,17.1965,0.157632,17.3195,0.225639,-9.99999e+08,-9.99999e+08,1

In [12]:
df_uk = tab_uk_sample.to_dataframe()
del tab_uk_sample

from astropy.coordinates import Angle
ra = Angle(df_uk['RA'].values,'rad').to('deg')
dec= Angle(df_uk['DEC'].values,'rad').to('deg')
df_uk['RA'] = ra
df_uk['DEC'] = dec

df_uk.describe()

           SOURCEID             RA            DEC        SIGRA       SIGDEC  \
count  2.313380e+05  231338.000000  231338.000000     231338.0     231338.0   
mean   4.338322e+11      21.215457       0.013751 -999999488.0 -999999488.0   
std    2.958270e+06       4.365869       0.389857          0.0          0.0   
min    4.338279e+11      13.597006      -0.721999 -999999488.0 -999999488.0   
25%    4.338291e+11      17.474631      -0.301291 -999999488.0 -999999488.0   
50%    4.338325e+11      21.748381       0.031961 -999999488.0 -999999488.0   
75%    4.338345e+11      24.765722       0.342271 -999999488.0 -999999488.0   
max    4.338373e+11      28.543990       0.703997 -999999488.0 -999999488.0   

               EPOCH            EBV             AY             AJ  \
count  231338.000000  231338.000000  231338.000000  231338.000000   
mean     2006.341197       0.032063       0.038828       0.028504   
std         0.630836       0.005339       0.006466       0.004747   
min      200

#### Clean table null values

I assume every negative value in *semi-positive* columns to be `Null/NaN`.
Such columns are the positional (RA,Dec) standard deviations and the magnitude columns.
(Physically speaking, magnitudes are/can be negative; It just does not happen in our case --galaxies.)

In [13]:
cols = ['SIGRA','SIGDEC']
cols.extend( [ c for c in df_uk.filter(like='MAG',axis=1).columns ] )
for col in cols:
    idx = df_uk[col] < 0
    df_uk.loc[idx,col] = None

In [14]:
ID_column_uk = 'SOURCEID'
df_uk[ID_column_uk] = df_uk[ID_column_uk].astype(int).astype(str)

df_uk.head()

,SOURCEID,RA,DEC,SIGRA,SIGDEC,EPOCH,EBV,AY,AJ,AH,...,HAPERMAG3,HAPERMAG3ERR,KHALLMAG,KHALLMAGERR,KPETROMAG,KPETROMAGERR,KPSFMAG,KPSFMAGERR,KAPERMAG3,KAPERMAG3ERR
0,433827929919,23.367877,0.687888,NaN,NaN,2005.902151,0.022051,0.026704,0.019603,0.012745,...,18.506187,0.151259,16.839336,0.166297,17.013243,0.244477,NaN,NaN,17.544065,0.124013
1,433827929920,23.460531,0.688232,NaN,NaN,2005.902151,0.023649,0.028639,0.021024,0.013669,...,16.893356,0.035704,16.618038,0.080816,16.661270,0.118318,NaN,NaN,16.565256,0.051986
2,433827929921,23.414002,0.687667,NaN,NaN,2005.902151,0.022683,0.027469,0.020165,0.013111,...,18.633087,0.170696,17.692490,0.398679,17.692490,0.398679,NaN,NaN,17.764069,0.152736
3,433827929922,23.280570,0.687302,NaN,NaN,2005.902151,0.022506,0.027254,0.020007,0.013008,...,16.557207,0.026067,15.337543,0.038435,15.085359,0.060233,NaN,NaN,15.807901,0.025575
4,433827929923,23.474184,0.687547,NaN,NaN,2005.902151,0.023927,0.028976,0.021271,0.013830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Plot table distributions

In [15]:
# from bokeh.io import output_notebook
# output_notebook()

In [16]:
# from booq.plot import PlotHisto
# p = PlotHisto()
# for col in ['YAPERMAG3','J_1APERMAG3','HAPERMAG3','KAPERMAG3']:
#     ds_label = p.add_dataset(df_uk,label=col,x=col)
#     p.plot(ds_label)
# p.show()

In [17]:
# from booq.plot import PlotHisto
# p = PlotHisto()
# for col in ['YAPERMAG3','J_1APERMAG3','HAPERMAG3','KAPERMAG3']:
#     ds_label = p.add_dataset(df_uk_sample,label=col,x=col)
#     p.plot(ds_label)
# p.show()

### Plot sky distribution

In [18]:
# from booq.plot import PlotScatter
# scatter = PlotScatter()

# # uk_label = scatter.add_dataset(cat_uk,x='RA',y='DEC',label='scatter_uk')
# # scatter.plot(uk_label)

# uks_label = scatter.add_dataset(df_uk,x='RA',y='DEC',label='scatter_uk_sample')
# scatter.plot(uks_label)

# lm_label = scatter.add_dataset(df_lm,x='RAdeg',y='DEdeg',label='scatter_lm')
# scatter.plot(lm_label)

# scatter.show()

### Radii definition

In [19]:
from sklearn.neighbors import NearestNeighbors

X = df_uk[['RA','DEC']].values
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
distances_uk, indices_uk = nbrs.kneighbors(X)
distances_uk = distances_uk.transpose()

In [20]:
X = df_lm[['RAdeg','DEdeg']].values
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
distances_lm, indices_lm = nbrs.kneighbors(X)
distances_lm = distances_lm.transpose()

In [21]:
# from booq.plot import PlotHisto
# p = PlotHisto()
# ds_label = p.add_dataset(distances_uk,label='1st-neighbour ukidss',x=1)
# p.plot(ds_label)
# ds_label = p.add_dataset(distances_lm,label='1st-neighbour lamassa',x=1)
# p.plot(ds_label)
# p.show()

##### Nearest neighbours

Let's take the nearest neighbours plot above as a guide to define the search radii when cross-matching. The typical distance among "LaMassa" (x-ray) sources -- *i.e.*, the peak of the distribution, is around 90 arcsec; beggining at $\approx 30"$. On "UKIDSS", if we consider the first peak, it is around $7.2"$.

We may then take $r_s = 7 arcsec$ as the search radius. If we sextuple this radius, the corresponding circle area would be 36 times bigger; we may consider such radius as the background area outer radius. By defining the inner and outer radii as $r_i = 7"$ and $r_o = 42 arcsec$ the annular region is 35 times the ancillary area, which should give us a good background sample.

In [22]:
from astropy.units import arcsec

rs = 7 * arcsec
ri = rs
ro = 6 * ri

### Cross-Match

Defined the corresponding radii for ancillary and background samples search, we proceed now with the corresponding cross-matchings as the first step of samples definition.

In [23]:
from booq.pipelines import mle

In [24]:
cols_A = {'ra':'RAdeg',
          'dec':'DEdeg',
          'id':'Seq',
          'pos_err':'e_Pos'}

cols = list(cols_A.values())

cat_A = df_lm[cols]

In [25]:
magnitude_columns = 'YAPERMAG3 J_1APERMAG3 HAPERMAG3 KAPERMAG3'.split()

feature_column = magnitude_columns[0]

cols_B = {'ra':'RA',
          'dec':'DEC',
          'id':'SOURCEID'}

cols = list(cols_B.values())

cols.append(feature_column)

cat_B = df_uk[cols]

In [26]:
from importlib import reload
reload(mle)

mle_match = mle.mle(cat_A, cat_B,
                    cols_A, cols_B,
                    feature_column,
                    ancillary_radius=rs,
                    background_radii=[ri,ro],
                    inner_join=False)


In [27]:
print(len(mle_match))
mle_match.head(20)

2862


A                                B                                  \
       RAdeg  DEdeg   Seq e_Pos      SOURCEID         RA       DEC  YAPERMAG3   
2359  14.097  0.166  2359   4.7          None        NaN       NaN        NaN   
2360  14.115 -0.353  2360   3.5  433834423123  14.114893 -0.352297  17.924706   
2361  14.115 -0.160  2361   5.8          None        NaN       NaN        NaN   
2362  14.142 -0.442  2362   3.4  433836362467  14.141094 -0.440343  18.888762   
2363  14.154 -0.448  2363   2.1  433836362494  14.154556 -0.447174  18.222660   
2364  14.162 -0.357  2364   5.2          None        NaN       NaN        NaN   
2365  14.162  0.038  2365   4.0  433832563084  14.162007  0.039508  18.878496   
2366  14.163 -0.400  2366   3.8  433836363120  14.164038 -0.399393        NaN   
2367  14.170 -0.244  2367   2.7          None        NaN       NaN        NaN   
2368  14.172 -0.400  2368   4.3  433836362527  14.171665 -0.398114  20.279314   
2369  14.186 -0.389  2369   4.4  433836363145  14.185372 -0.389149        NaN   
2370  14.196 -0.166  2370   3.6  433834437127  14.195810 -0.164875  20.141270   
2371  14.197 -0.577  2371   3.4  433836363155  14.196392 -0.576760        NaN   
2372  14.207  0.456  2372   4.7  433829058905  14.206796  0.455432  20.627474   
2373  14.215 -0.012  2373   1.3  433832563245  14.215556 -0.011784  18.467182   
2374  14.217  0.068  2374   3.4  433832563006  14.217423  0.068694  20.095844   
2375  14.231 -0.452  2375   3.1  433836363033  14.231218 -0.451879        NaN   
2376  14.234 -0.034  2376   2.9          None        NaN       NaN        NaN   
2377  14.239  0.147  2377   2.9  433830897005  14.239424  0.147334  20.111305   
2378  14.239  0.004  2378   4.6          None        NaN       NaN        NaN   

     AB_YAPERMAG3                                                           
      Reliability        LR    duplicates    duplicates_LR    duplicates_R  
2359          NaN       NaN          None             None            None  
2360     1.000000  0.073829          None             None            None  
2361          NaN       NaN          None             None            None  
2362     1.000000  0.019703          None             None            None  
2363     1.000000  0.070513          None             None            None  
2364          NaN       NaN          None             None            None  
2365     1.000000  0.033257          None             None            None  
2366          NaN       NaN          None             None            None  
2367          NaN       NaN          None             None            None  
2368     1.000000  0.012236          None             None            None  
2369          NaN       NaN          None             None            None  
2370     1.000000  0.036455          None             None            None  
2371          NaN       NaN          None             None            None  
2372     1.000000  0.023225          None             None            None  
2373     1.000000  0.078296          None             None            None  
2374     0.586907  0.047625  433832563005  0.0335210589771  0.413093071155  
2375          NaN       NaN          None             None            None  
2376          NaN       NaN          None             None            None  
2377     1.000000  0.055311          None             None            None  
2378          NaN       NaN          None             None            None

In [28]:
# mle_match.to_csv('mle_match.csv')